In [19]:
import pandas as pd
import xgboost as xg 
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from typing import List
from sklearn.preprocessing import StandardScaler

In [20]:
df = pd.read_csv('Crop_recommendation.csv')

df

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
1692,117,86,48,28.695620,82.541958,6.225225,116.161684,banana
1693,114,94,53,26.335449,76.853201,6.190757,118.685826,banana
1694,110,78,50,25.937302,78.898644,5.915569,98.217475,banana
1695,94,70,48,25.136865,84.883944,6.195152,91.464425,banana


In [21]:
df['temperature'] = df['temperature'].astype(int)
df['humidity'] = df['humidity'].astype(int)
df['ph'] = df['ph'].astype(int)
df['rainfall'] = df['rainfall'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   N            1697 non-null   int64 
 1   P            1697 non-null   int64 
 2   K            1697 non-null   int64 
 3   temperature  1697 non-null   int64 
 4   humidity     1697 non-null   int64 
 5   ph           1697 non-null   int64 
 6   rainfall     1697 non-null   int64 
 7   label        1697 non-null   object
dtypes: int64(7), object(1)
memory usage: 106.2+ KB


In [22]:
df['label'].unique()

array(['rice', 'maize', 'Soyabeans', 'beans', 'peas', 'groundnuts',
       'cowpeas', 'banana', 'mango', 'grapes', 'watermelon', 'apple',
       'orange', 'cotton', 'coffee'], dtype=object)

In [23]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

df['label']

scaler = StandardScaler()


In [24]:
# print(le.classes_)
from typing import List

crops: List = ['Soyabeans', 'apple', 'banana', 'beans', 'coffee', 'cotton', 'cowpeas', 'grapes',
 'groundnuts', 'maize', 'mango', 'orange', 'peas', 'rice', 'watermelon']

crops_dict = dict(enumerate(crops))
crops_dict

{0: 'Soyabeans',
 1: 'apple',
 2: 'banana',
 3: 'beans',
 4: 'coffee',
 5: 'cotton',
 6: 'cowpeas',
 7: 'grapes',
 8: 'groundnuts',
 9: 'maize',
 10: 'mango',
 11: 'orange',
 12: 'peas',
 13: 'rice',
 14: 'watermelon'}

In [25]:
x = df.drop('label',axis=1)
y = df['label']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,shuffle=True)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1527, 7)
(170, 7)
(1527,)
(170,)


In [37]:
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [45]:
crop_xg_model = xg.XGBClassifier(
    objective='multi:softprob',
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
)

crop_xg_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [46]:
model_accuracy = 100 * crop_xg_model.score(x_test, y_test)

print(f'The model has an accuracy of {model_accuracy:.2f}%')

The model has an accuracy of 100.00%


In [50]:
import numpy as np

vals = [[70, 72, 68, 23, 45, 6, 140]]
# vals = [[100 ,89 ,88, 22,75, 7,	55]]

instance = scaler.transform(vals)  # type: ignore

predictions = crop_xg_model.predict_proba(instance)

# sample_crops = le.inverse_transform(predictions)

pred_class = np.argmax(predictions)
certainty = 100 * np.max(predictions)

print(f"Class: {crops_dict[pred_class]}")
print(f"{certainty:.2f}%")

Class: banana
47.78%


In [ ]:
# Save model

import joblib

joblib.dump(crop_xg_model, 'crop_model.pkl')

['crop_model.pkl']

In [52]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15, random_state=0)

RF.fit(x_train, y_train)
rf_predictions = RF.predict(x_test)
print(classification_report(y_test,rf_predictions))

RandomForestClassifier(n_estimators=15, random_state=0)

In [57]:
data = [[50,28, 30, 34, 50, 7, 140]]
data = np.array(data)

pred = RF.predict()
pred

array([1])

In [ ]:
joblib.dump(RF, 'RF_model.pkl')
